此份代码用于SIT小组成员调试生成数据集，主要方法已调试好，可方便地在本地运行改进。

In [1]:
#引入所有需要的模块，如有未安装的库建议pip install
# PyAutoGUI是一个纯Python的GUI自动化工具，其目的是可以用程序自动控制鼠标和键盘操作，多平台支持（Windows，OS X，Linux）。
#cv2即opencv2，用于实现电脑摄像头的调用等操作
#PIL 为Python 常用图像处理模块
import pyautogui as pag
import time
import requests
import os
import cv2
import urllib3,base64
import json
from PIL import Image
from urllib.parse import urlencode

### Part 1 收集摄像头照片
通过openCV调用摄像头，在按下空格键时保存摄像头照片，命名为鼠标所在位置的元组形式。

In [ ]:
#将此处改为初始数据集的保存路径
save_path = 'D:/UserData/Documents/Jupyter/EyeTrack/DataSet/'
#获取屏幕大小
screenWidth,screenHeight = pag.size()
#计算获得屏幕原点位置，定为屏幕中心点
origin = (screenWidth/2,screenHeight/2)

cap=cv2.VideoCapture(0) #调用摄像头，0为电脑自带摄像头，1为外部摄像头
while(1):
    ret,frame = cap.read()
    k=cv2.waitKey(1)
    if k==27: #Esc键退出
        break
    elif k==32:#空格键保存图片
        #获取当前鼠标绝对位置
        currMouseX,currMouseY = pag.position() 
        #将鼠标绝对位置转化为相对于原点的相对坐标，并进行归一化处理
        #对归一化处理有疑惑可参考：https://www.jianshu.com/p/95a8f035c86c
        currMouse = ((currMouseX-origin[0])/origin[0],(origin[1]-currMouseY)/origin[1])
        
        cv2.imwrite(save_path+str(currMouse)+'.jpg',frame)
        print(currMouse) #方便调试，若成功保存则会立刻在控制台输出
    cv2.imshow("capture", frame)
cap.release()
cv2.destroyAllWindows()

### Part 2 裁剪获取眼部图片
通过BaiduAI的人脸识别功能获取脸部特征，并根据所需取的脸部特征点，应用Pillow的crop方法进行裁剪，相关参考文档如下：
1. BaiduAI 人脸识别技术文档：https://ai.baidu.com/ai-doc/FACE/yk37c1u4t
2. Pillow 技术文档：https://pillow.readthedocs.io/en/stable/reference/Image.html

In [2]:
#BaiduAI 发送请求的必备信息，可自行创建应用并修改
access_token = '24.4c30782b2563641d533c5791a7d0193a.2592000.1615813055.282335-22192901'

In [3]:
#BaiduAI 人脸识别方法调用，参数为文件路径，返回BaiduAI平台的反馈
def BaiduMethod(filepath):
    file = open(filepath,'rb')
    img = open(filepath,'rb')
    #参数images：图像base64编码 分别base64编码后的2张图片数据，需urlencode，半角逗号分隔，单次请求最大不超过20M
    img1 = base64.b64encode(img.read())
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/detect"+ "?access_token=" + access_token
    params = {'image':str(img1,'utf-8'),'image_type':'BASE64','face_field':'landmark'}
    headers = {'content-type': 'application/json'}
    return requests.post(request_url, data=params, headers=headers)

以单个文件为例，方便数据集收集方法的调试。

In [4]:
#此处改为所想读取的单个图片
filepath = 'D:/UserData/Documents/Jupyter/EyeTrack/DataSet/(-0.5206666666666667, 0.013).jpg'
#考虑到百度AI平台的对并发量的限制，此处推荐分开处理，即获取response后不再重复对同一张照片进行处理
response = BaiduMethod(filepath)
if response:
    print (response.json())

{'error_code': 0, 'error_msg': 'SUCCESS', 'log_id': 1018415991010, 'timestamp': 1613221365, 'cached': 0, 'result': {'face_num': 1, 'face_list': [{'face_token': '7ceb0c9266de70bdb7e56d4f20f9f650', 'location': {'left': 277.74, 'top': 66.96, 'width': 197, 'height': 209, 'rotation': 2}, 'face_probability': 1, 'angle': {'yaw': -3.76, 'pitch': -6.47, 'roll': -0.05}, 'landmark': [{'x': 332.14, 'y': 109.03}, {'x': 426.11, 'y': 113.61}, {'x': 381.88, 'y': 149.83}, {'x': 376.32, 'y': 211.38}], 'landmark72': [{'x': 273.85, 'y': 137.52}, {'x': 275, 'y': 168.56}, {'x': 278.64, 'y': 199.33}, {'x': 286.85, 'y': 230}, {'x': 309.34, 'y': 259.37}, {'x': 340.03, 'y': 276.43}, {'x': 371.56, 'y': 281.08}, {'x': 401.48, 'y': 277.06}, {'x': 431.51, 'y': 260.88}, {'x': 452.94, 'y': 233.46}, {'x': 462.94, 'y': 204.26}, {'x': 468.68, 'y': 174.71}, {'x': 471.56, 'y': 145.52}, {'x': 310.94, 'y': 113.34}, {'x': 321.13, 'y': 105.67}, {'x': 332.19, 'y': 103.28}, {'x': 342.57, 'y': 105.49}, {'x': 352.52, 'y': 113.46}

### 声明
图片切割必须解决如下问题：
眼部切割所得的图片必须是同样大小的矩形，如何实现？
1. 考虑提前定长（如何确保一定包含眼部？不同角度，位置的照片中眼部面积可能不同）
2. 根据设备调整模型？

暂定解决方案：先完整截取眼部图片，随后进行微调尽量删除无效信息，其次通过resize方法调整图片大小至模型所需输入参数。

In [6]:
landmark = response.json()['result']['face_list'][0]['landmark72']
img2 = Image.open(filepath)
#截取方法待调整
box = (landmark[22]['x'],landmark[22]['y'],landmark[22]['x']+150,landmark[22]['y']+50)
img3 = img2.crop(box)
img3.show() #实时显示切割图片方便微调

在调试完眼部图片截取方法后运行该下面模块进行以获取眼部图片数据集。

In [7]:
path = 'D:/UserData/Documents/Jupyter/EyeTrack/DataSet/' #待遍历目录路径
save_path = 'D:/UserData/Documents/Jupyter/EyeTrack/DataSet02/'#保存文件目录
dirs = os.listdir(path)
for file in dirs:
    type(file)
    break
    filepath = os.path.join(path,file)
    response = BaiduMethod(filepath)
    if response and response.json()['error_code']==0:#成功收到且未报错
        landmark = response.json()['result']['face_list'][0]['landmark72']
        img2 = Image.open(filepath)
        #截取方法待调整
        box = (landmark[22]['x'],landmark[22]['y'],landmark[22]['x']+150,landmark[22]['y']+50)
        img3 = img2.crop(box)
        img3.save(save_path+file+'.jpg')

In [12]:
print(file)

(0.9993333333333333, 1.0).jpg


In [17]:
rows=[]
rows.append([1,2,'3'])

In [18]:
print(rows)

[[1, 2, '3']]
